In [1]:
!pip install moviepy torch transformers spacy deep-translator openai-whisper googletrans==4.0.0-rc1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
! python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 29.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os
import warnings
import logging
import torch
import moviepy.editor as mp
from transformers import pipeline
from googletrans import Translator
import spacy
from deep_translator import GoogleTranslator
import whisper

  if event.key is 'enter':



In [4]:

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers.pipelines.token_classification')
warnings.filterwarnings("ignore", category=UserWarning, module='transformers.modeling_utils')

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load models
logging.info("Loading models...")
WHISPER_MODEL = whisper.load_model("base")  # Use "small", "medium", or "large" for better accuracy
PUNCTUATION_MODEL = pipeline("text2text-generation", model="oliverguhr/fullstop-punctuation-multilang-large")
SUMMARIZATION_MODEL = pipeline("summarization", model="facebook/bart-large-cnn")
NER_MODEL = spacy.load("en_core_web_trf")  # Use spaCy's transformer-based NER model

# Functions for the pipeline

def video_to_audio(in_path, out_path):
    """Convert video file to audio file using moviepy."""
    try:
        logging.info(f"Converting video to audio: {in_path} -> {out_path}")
        video = mp.VideoFileClip(in_path)
        video.audio.write_audiofile(out_path)
        logging.info("Video-to-audio conversion completed.")
    except Exception as e:
        logging.error(f"Error converting video to audio: {e}")
        raise

def speech_to_text(audio_path):
    """Convert audio to text using OpenAI's Whisper."""
    try:
        logging.info(f"Transcribing audio: {audio_path}")
        result = WHISPER_MODEL.transcribe(audio_path)
        text = result["text"]
        logging.info("Speech-to-text conversion completed.")
        return text
    except Exception as e:
        logging.error(f"Error transcribing audio: {e}")
        raise

def restore_punctuation(text):
    """Restore punctuation in text using a pre-trained model."""
    try:
        logging.info("Restoring punctuation...")
        punctuated_text = PUNCTUATION_MODEL(text)[0]["generated_text"]
        logging.info("Punctuation restoration completed.")
        return punctuated_text
    except Exception as e:
        logging.error(f"Error restoring punctuation: {e}")
        return text  # Fallback to original text

def translate_text(text, lang):
    """Translate text to a given language using DeepL or Google Translate."""
    try:
        logging.info(f"Translating text to {lang}...")
        translated_text = GoogleTranslator(source="auto", target=lang).translate(text)
        logging.info("Translation completed.")
        return translated_text
    except Exception as e:
        logging.error(f"Error translating text: {e}")
        return text  # Fallback to original text

def summarize_text(text):
    """Summarize text using a pre-trained BART model."""
    try:
        logging.info("Summarizing text...")
        summary = SUMMARIZATION_MODEL(text, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
        logging.info("Summarization completed.")
        return summary
    except Exception as e:
        logging.error(f"Error summarizing text: {e}")
        return text  # Fallback to original text

def named_entity_recognition(text):
    """Perform Named Entity Recognition (NER) using spaCy."""
    try:
        logging.info("Performing Named Entity Recognition...")
        doc = NER_MODEL(text)
        entities = [
            {
                "Entity": ent.text,
                "Entity Group": ent.label_,
                "Start Position": ent.start_char,
                "End Position": ent.end_char,
            }
            for ent in doc.ents
        ]
        logging.info("NER completed.")
        return entities
    except Exception as e:
        logging.error(f"Error performing NER: {e}")
        return []

# Main script
if __name__ == "__main__":
    try:
        # Upload the video file (for Google Colab)
        from google.colab import files
        uploaded = files.upload()
        video_path = list(uploaded.keys())[0]

        # Convert video to audio
        audio_path = "sample_audio.wav"
        video_to_audio(video_path, audio_path)

        # Convert audio to text
        text = speech_to_text(audio_path)

        # Restore punctuation
        text = restore_punctuation(text)

        # Translation
        language = input("Choose a language (fr for French, es for Spanish, ar for Arabic): ")
        translated_text = translate_text(text, language)

        # Summarization
        summary = summarize_text(translated_text)

        # Named Entity Recognition
        entities = named_entity_recognition(translated_text)

        # Print results
        print("Original Text:", text)
        print("Translated Text:", translated_text)
        print("Summary:", summary)
        print("Named Entities:")
        for entity in entities:
            print(f"Entity: {entity['Entity']}, Group: {entity['Entity Group']}, "
                  f"Start: {entity['Start Position']}, End: {entity['End Position']}")

        # Save results to file
        with open("result.txt", "w") as f:
            f.write(f"Original Text:\n{text}\n\n")
            f.write(f"Translated Text ({language}):\n{translated_text}\n\n")
            f.write(f"Summary:\n{summary}\n\n")
            f.write("Named Entities:\n")
            for entity in entities:
                f.write(f"Entity: {entity['Entity']}, Group: {entity['Entity Group']}, "
                        f"Start: {entity['Start Position']}, End: {entity['End Position']}\n")

    except Exception as e:
        logging.error(f"An error occurred in the main script: {e}")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 93.7MiB/s]
  checkpoint = torch.load(fp, map_location=device)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

MoviePy - Done.


ERROR:root:Error restoring punctuation: 'Text2TextGenerationPipeline' object has no attribute 'prefix'


Choose a language (fr for French, es for Spanish, ar for Arabic): ar


ERROR:root:Error summarizing text: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

  with torch.cuda.amp.autocast(self._mixed_precision):



Original Text:  Welcome to our presentation on optimizing the traveling salesman problem using parallelism techniques in open MP. We'll explore how parallel computing can enhance solutions to this classic optimization challenge. The traveling salesman problem or TSP is a well known optimization challenge. The goal is to find the shortest route that visits each city exactly once and returns to the starting point. This problem is fundamental in fields like logistics and planning. TSP has several practical variants such as dynamic TSP for real time routing and the vehicle routing problem for logistics. These variants demonstrate TSP's versatility in addressing complex real world challenges. TSP is a classic optimization challenge with significant real world importance. It spans applications and logistics, robotics and communication networks. However, its complexity grows exponentially classifying it as MP hard. The complexity of TSP makes brute force methods impractical. Evaluating all pe